In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv

load_dotenv("../.env")

True

In [2]:
import mlflow
import torch
from prisma import Prisma
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

device

device(type='cuda')

In [4]:
model_uri = f"runs:/5261022518c9417692ab0d3315ffb9e0/such-toxic"
sentence_transformer_model = "sentence-transformers/all-MiniLM-L6-v2"
ral_e_dataset = "../datasets/retrain_reddit_abuse_test.txt"
batch_size = 100

In [5]:
sentence_transformer = SentenceTransformer(sentence_transformer_model, device=device)
such_toxic = mlflow.pytorch.load_model(model_uri).to(device)


def classifiy_batch(batch):
    embeddings = sentence_transformer.encode(batch, convert_to_tensor=True)
    out = such_toxic(embeddings).cpu().detach().numpy().tolist()
    out = zip(batch, out)
    out = map(
        lambda x: {
            "comment_text": x[0],
            "toxic": x[1][0],
            "severe_toxic": x[1][1],
            "obscene": x[1][2],
            "threat": x[1][3],
            "insult": x[1][4],
            "identity_hate": x[1][5],
        },
        out,
    )
    return list(out)


classifiy_batch(["Hello", "World"])

[{'comment_text': 'Hello',
  'toxic': 0.01214311458170414,
  'severe_toxic': 0.0004117148055229336,
  'obscene': 0.00260039116255939,
  'threat': 0.00035658563137985766,
  'insult': 0.003171957330778241,
  'identity_hate': 0.0006483225733973086},
 {'comment_text': 'World',
  'toxic': 0.06567952781915665,
  'severe_toxic': 0.0016220887191593647,
  'obscene': 0.027771923691034317,
  'threat': 0.001004645018838346,
  'insult': 0.008807024918496609,
  'identity_hate': 0.0010928488336503506}]

In [6]:
db = Prisma()
db.connect()


def process_batch(batch):
    out = classifiy_batch(batch)
    for item in out:
        db.comments.create(item)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
with open(ral_e_dataset, "r") as file:
    batch = []
    for line in tqdm(file):
        batch.append(line)
        if len(batch) == 100:
            process_batch(batch)
            batch = []
    if batch:
        process_batch(batch)

14935it [04:41, 53.01it/s]


In [8]:
db.disconnect()